<div class="jumbotron jumbotron-fluid">
  <div class="container">
    <h1 class="display-4">Base Jupyter Notebook</h1>
    <p class="lead">Examples on how to load data from different sources, in different formats.</p>
  </div>
</div>

<a href="https://colab.research.google.com/github/BL-Labs/Jupyter-notebooks-projects-using-BL-Sources/blob/master/Microsoft19thCenturyBooks/base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### In this Notebook:

 * Load metadata inside a CSV / TSV file (compressed inside a zip file), local or downloadable from a URL
 * Count records with a given value in a certain field:
     e.g., number of records by publication date (year);
 * Plot the those values in a Bar Chart;
 * Generate a Word Cloud of the top 20 Places of Publication;
 * Load metadata from a JSON file (compressed inside a zip file)
 * Generate a Word Cloud of Tags popularity (970,204 tags)
 * Access and read a big file .BZ2 compessed (11.4GiB)  file from data.bl.uk or from BL's Research Repository (https://bl.iro.bl.uk/)
 
 
 
 #### Notes:

     * If using BINDER or a did a git clone to your Jupyter Notebook LOCAL SERVER, if you want to clean the output from the previous run, from the main menu, choose "Kernel" > "Restart and Clear Output"; if using Google Colab, please open instead  "base_demo_partial-run.ipynb" notebook;
     * If using BINDER or a did a git clone to your Jupyter Notebook LOCAL SERVER, run cell 1, skip cell 2, place the cursor on cell 3 and from the main menu, choose "Cell" > "Run All Bellow", if not executing cell-by-cell.
     * If using Google Colab, run cell 1 and 2, skip cell 3, place the cursor on cell 4 and from the main menu, choose "Runtime" > "Run after", if not executing cell-by-cell.
 

In [ ]:
# imports

import pandas as pd
# import json, os, shutil, codecs
import zipfile
import io
import urllib
import json
import altair as alt

### Let us first have a look at metadata inside a **CSV / TSV file (small one)**:

In [ ]:
"""
==> ONLY RUN this CELL if using Google COLAB (see bellow for BINDER or a LOCAL Jupyter Notebooks SERVER)
"""
# read metadata file #1 -- small TSV file

filename = "https://github.com/BL-Labs/Jupyter-notebooks-projects-using-BL-Sources/raw/master/Microsoft19thCenturyBooks/data/Microsoft-Books.zip"
df = pd.read_csv(filename, delimiter="\t", compression ="zip")

In [ ]:
"""
==> ONLY RUN this CELL if using BINDER or a LOCAL Jupyter Notebooks SERVER 
"""
# read metadata file #1 -- small TSV file

filename = "data/Microsoft-Books.zip"
df = pd.read_csv(filename, delimiter="\t", compression ="zip")

In [ ]:
# number of records inside this file / dataframe

df.size

In [ ]:
# 1st 3 records

df.head(3)

In [ ]:
df["Content type"].value_counts()

In [ ]:
# Top of Publication Dates

df["Date of publication (standardised)"].value_counts()

In [ ]:
# ploting them with the default sort of the value counts -- not the best
df["Date of publication (standardised)"].value_counts().plot(kind='bar')

In [ ]:
# another way to calculate the count of the number of records per date of publication -- this time sorting by value of the field, instead of the default counts value   

CountPerYear = pd.value_counts(df["Date of publication (standardised)"].values, sort=False).sort_index()
# equivalent to > CountPerYear = df["Date of publication (standardised)"].value_counts().sort_index()

CountPerYear

In [ ]:
# ploting it now it allows to have a better view of the evolution over the years -- peak value: year 1897 = 1491 entries

CountPerYear.plot.bar(figsize=(100,20))

In [ ]:
# Density plot (?)

CountPerYear.plot.kde()

In [ ]:
# top of Genre

df["Genre"].value_counts()

In [ ]:
# top of Literary form

df["Literary form"].value_counts()

In [ ]:
# Top of Country of Publication

df["Country of publication"].value_counts()

In [ ]:
# top of Place of Publication

df["Place of publication"].value_counts()

In [ ]:
# How many have that info

df["Place of publication"].size

In [ ]:
# Generate a Word Cloud of the top 20 Places of Publication

!pip install git+https://git@github.com/kavgan/word_cloud.git

from word_cloud.word_cloud_generator import WordCloud
from IPython.core.display import HTML
 
list_of_documents=[]
 
for i in range (df["Place of publication"].size):
    list_of_documents.append(str(df["Place of publication"][i]))

# initialize WordCloud
wc=WordCloud()

embed_code=wc.get_embed_code(text=list_of_documents,random_color=True,topn=20)

HTML(embed_code)

In [ ]:
# Words appearing in the 49,316 titles: build the list of documents

list_of_documents=[]
 
for i in range (df["Title"].size):
    list_of_documents.append(str(df["Title"][i])) 

In [ ]:
# Word Cloud: top 100, using the module above (from GtHub)
    
embed_code=wc.get_embed_code(text=list_of_documents,random_color=True,topn=100)

HTML(embed_code)

In [ ]:
# using another module: wordcloud (standard, built-in)

#from wordcloud import WordCloud, STOPWORDS
from wordcloud import WordCloud
import matplotlib.pyplot as plt


STOP_WORDS = frozenset([
    "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side",
    "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such",
    "system", "take", "ten", "than", "that", "the", "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither",
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "said","you", "your", "yours", "yourself",
    "yourselves", "etc'","und"])


wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'black',
    stopwords = STOP_WORDS).generate(str(list_of_documents))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()


### Let us check another metadata set inside a **JSON file (big one)**:


In [ ]:
# RUN this CELL if using BINDER or a LOCAL Jupyter Notebooks SERVER (see bellow for Google Colab)
# read metadata file #2 (big JSON file -- book_data.json (52MB) inside book_data.zip (13MB))

filename = "data/book_data.zip"

# unzips the files inside book_data.zip
z = zipfile.ZipFile(filename)

In [ ]:
# RUN this CELL if using Google COLAB
# read metadata file #2 (big JSON file -- book_data.json (52MB) inside book_data.zip (13MB))

zip_file = "https://github.com/BL-Labs/Jupyter-notebooks-projects-using-BL-Sources/raw/master/Microsoft19thCenturyBooks/data/book_data.zip"

# OR test loading it from data.bl.uk
# zip_file = "https://data.bl.uk/digbks/book_data.zip"

access_url = urllib.request.urlopen(zip_file)

# unzips the files inside book_data.zip
z = zipfile.ZipFile(io.BytesIO(access_url.read()))

In [ ]:
# loads JSON from the first (only) file inside into a list 
meta = json.loads(z.read(z.infolist()[0]).decode('utf-8'))


In [ ]:
# number of records inside this list

len(meta)

In [ ]:
# checking a ramdom record (#342)

meta[342]

In [ ]:
# By Andrew Jackson (UKWA / BL):

years = []
pages = []
volumes = []
titles = []
publishers = []
place = []
url = []
shelfmarks = []
no_date = []
no_pages = []

# Loop over the books:
for book in meta:
    # Get the year
    date = book['date']
    if date == '':
        no_date.append(book)
        continue
    # Get the volumes as the number of image groups:
    vols = book.get('imgs',[])
    # Loop though the image groups and count the images
    p = 0
    for v in vols:
        p += len(vols[v])
    # Skip ones with no pages:
    if p == 0:
        no_pages.append(book)
        continue
    # This is a 'good one' so record it:
    years.append(int(date))
    volumes.append(len(vols))
    pages.append(p)
    # and titles:
    title = book.get('title','')
    title = ''.join(title)[:100]
    titles.append(title)
    # publisher
    publishers.append(book.get('publisher', ""))
    # place
    place.append(book.get('place',''))
    # url
    url.append(' '.join(list(book.get('pdf',{}).values())))
    shelfmarks.append(', '.join(book['shelfmarks']))

source = pd.DataFrame({
    'year': years,
    'pages': pages,
    'volumes': volumes,
    'title': titles,
    'publisher': publishers,
    'place': place,
    'ids': url,
    'shelfmarks': shelfmarks
})
source

In [ ]:
# By Andrew Jackson (UKWA / BL):

alt.data_transformers.disable_max_rows()
#alt.renderers.enable('notebook')

# define selection
click = alt.selection_multi(encodings=['color'])

chart = alt.Chart(source).mark_point(
    filled=True
).encode(
    alt.X('year:Q',
        scale=alt.Scale(zero=False)
    ),
    y='pages:Q',
    size='volumes:N',
    color='volumes:N',
#    href='url:N',
    tooltip=['title:N', 'volumes:N', 'pages:N','year:N', 'publisher:N', 'place:N', 'shelfmarks:N'],
    opacity=alt.value(0.5)
).transform_filter(
    click
).properties(
    width=800,
    height=600,
    selection=click,
)

chart.save("digbks-year-pages-volumes-scatter.html")

chart

### Let us check other metadata set inside **CSVs files (including a big one)**:

In [ ]:
# read metadata file #3 (not MS Books, though -- big CSV file -- sherlock_flickr_tags.csv (27MB) + sherlocknet_tags_verbose.csv (207MB) inside sherlocknet_tags_csv.zip (44MB)), from BL Research Repository (British Library Labs: Projects | SherlockNet data, https://bl.iro.bl.uk/work/3205f8c5-f7dd-46fa-8bf4-9ac72f6fbb55)

zip_file = "https://s3.eu-west-2.amazonaws.com/importer.oar.bl.uk/bl/4ccd707f-eab9-4514-9fc3-17633274b17a"

access_url = urllib.request.urlopen(zip_file)

# unzips the files inside sherlocknet_tags_csv.zip
z = zipfile.ZipFile(io.BytesIO(access_url.read()))

In [ ]:
# loads metadata from the sherlock_flickr_tags.csv (27MB) CSV file into a 1st dataframe
df1 = pd.read_csv(z.open('sherlock_flickr_tags.csv'))

In [ ]:
# number of records inside this CSV / dataframe df1

df1.size

In [ ]:
# 1st 3 records

df1.head(3)

In [ ]:
# what are the most popular tags?

df1['tag'].value_counts()

In [ ]:
df1['tag'].size

In [ ]:
# Generate a Word Cloud of Tags popularity -- reads the tags whole 970,204 tags
 
list_of_documents=[]
 
for i in range (df1['tag'].size):
    list_of_documents.append(str(df1['tag'][i])) 
    
embed_code=wc.get_embed_code(text=list_of_documents,random_color=True,topn=100)

HTML(embed_code)

In [ ]:
# loads metadata from the sherlocknet_tags_verbose.csv (207MB) CSV file into a 2nd dataframe

df2 = pd.read_csv(z.open('sherlocknet_tags_verbose.csv'))

In [ ]:
# number of records inside this 2nd CSV / dataframe df2
df2.size

In [ ]:
# 1st 3 records

df2.head(3)

In [ ]:
# what are the most popular tags in this dataset? Compare if they match df1's

df2['tag'].value_counts()

In [ ]:
# How about reading a 11.4GiB .BZ2 file from data.bl.uk or from https://bl.iro.bl.uk/? Inside there's a tar file of 44GB -- and inside this the 65,277 JSON files)
# page level JSON text format from 45,455 digitised books (65,277 volumes) 

import bz2

# bz2_file = "https://data.bl.uk/digbks/dig19cbooksjsontext.bz2"
# bz2_file = "https://s3.eu-west-2.amazonaws.com/importer.oar.bl.uk/bl/0623cab5-58d7-4eba-b88b-bb32b86862d8"

access_url = urllib.request.urlopen(bz2_file)

# unzips the files inside
f = bz2.BZ2File(access_url, mode='r', buffering=None, compresslevel=9)

In [ ]:
# check the bz2 file
f.peek()

In [ ]:
# Inside the bz2 there's a tar file of 44GB
tar_file = f.read()